# Registry Pattern for Plugin Systems
### Plugin Interface:
- We define a common interface (Plugin in this case) that all plugins must implement. This ensures a consistent way to interact with plugins.
### Plugin Registry:
- The PluginRegistry class acts as a central repository for all plugins.
- It provides methods to register plugins, retrieve plugins, and execute plugins by name.
### Plugin Registration:
- Plugins are registered with the registry using a unique name or identifier.
- The registration process can include validation to ensure that only proper plugin instances are added.
### Dynamic Loading:
- Plugins can be loaded dynamically at runtime, allowing for easy extension of the application without modifying its core code.
- In a more advanced system, this could involve scanning directories for plugin files and loading them automatically.
### Execution:
- The registry provides a method to execute plugins by name, abstracting away the details of how plugins are stored and retrieved.
### Discoverability:
- The registry can provide methods to list available plugins, making it easy for the application or user to know what functionality is available.
## Advanced Concepts:
- Plugin Dependencies: The registry can manage dependencies between plugins, ensuring they are loaded in the correct order.
- Plugin Configuration: Extend the system to allow plugins to have their own configuration options.
- Hot-swapping: Implement functionality to update or replace plugins without restarting the application.
- Sandboxing: Implement security measures to run plugins in a controlled environment to prevent malicious behavior.
- Event System: Incorporate an event system where plugins can subscribe to and publish events, allowing for more complex interactions.

In [1]:
# Registry to manage plugins

from abc import ABC, abstractmethod

class Plugin(ABC):
    @abstractmethod
    def execute(self):
        pass

class PluginRegistry:
    def __init__(self):
        self._plugins = {}

    def register(self, name, plugin):
        if not isinstance(plugin, Plugin):
            raise ValueError("Plugin must inherit from Plugin class")
        self._plugins[name] = plugin

    def get_plugin(self, name):
        return self._plugins.get(name)

    def execute_plugin(self, name):
        plugin = self.get_plugin(name)
        if plugin:
            plugin.execute()
        else:
            print(f"Plugin '{name}' not found")

    def list_plugins(self):
        return list(self._plugins.keys())

# Example plugins
class HelloPlugin(Plugin):
    def execute(self):
        print("Hello from HelloPlugin!")

class DataProcessorPlugin(Plugin):
    def execute(self):
        print("Processing data...")

# Application using the plugin system
class Application:
    def __init__(self):
        self.plugin_registry = PluginRegistry()

    def load_plugins(self):
        self.plugin_registry.register("hello", HelloPlugin())
        self.plugin_registry.register("data_processor", DataProcessorPlugin())

    def run(self):
        print("Available plugins:", self.plugin_registry.list_plugins())
        self.plugin_registry.execute_plugin("hello")
        self.plugin_registry.execute_plugin("data_processor")

# Usage
app = Application()
app.load_plugins()
app.run()

Available plugins: ['hello', 'data_processor']
Hello from HelloPlugin!
Processing data...


In [5]:
# Managing dependencies

from abc import ABC, abstractmethod

class Plugin(ABC):
    @abstractmethod
    def execute(self):
        pass

class PluginRegistry:
    def __init__(self):
        self._plugins = {}
        self._dependencies = {}

    def register(self, name, plugin, dependencies=None):
        if not isinstance(plugin, Plugin):
            raise ValueError("Plugin must inherit from Plugin class")
        self._plugins[name] = plugin
        if dependencies:
            self._dependencies[name] = dependencies

    def get_plugin(self, name):
        return self._plugins.get(name)

    def execute_plugin(self, name):
        if name in self._dependencies:
            for dep in self._dependencies[name]:
                self.execute_plugin(dep)
        plugin = self.get_plugin(name)
        if plugin:
            plugin.execute()
        else:
            print(f"Plugin '{name}' not found")

    def list_plugins(self):
        return list(self._plugins.keys())

# Example plugin implementations
class DatabasePlugin(Plugin):
    def execute(self):
        print("Initializing database connection...")

class LoggerPlugin(Plugin):
    def execute(self):
        print("Setting up logging...")

class UserManagerPlugin(Plugin):
    def __init__(self, registry):
        self.registry = registry

    def execute(self):
        print("User Manager is starting...")
        # Here we're simulating the use of the database and logger
        # In a real scenario, you might get these plugins and use their methods
        print("Using database to fetch users...")
        print("Logging user activity...")

# Application using the plugin system
class Application:
    def __init__(self):
        self.plugin_registry = PluginRegistry()

    def load_plugins(self):
        self.plugin_registry.register("database", DatabasePlugin())
        self.plugin_registry.register("logger", LoggerPlugin())
        self.plugin_registry.register("user_manager", UserManagerPlugin(self.plugin_registry), 
                                      dependencies=["database", "logger"])

    def run(self):
        print("Available plugins:", self.plugin_registry.list_plugins())
        print("\nExecuting UserManager plugin (which depends on Database and Logger):")
        self.plugin_registry.execute_plugin("user_manager")

# Usage
app = Application()
app.load_plugins()
app.run()

Available plugins: ['database', 'logger', 'user_manager']

Executing UserManager plugin (which depends on Database and Logger):
Initializing database connection...
Setting up logging...
User Manager is starting...
Using database to fetch users...
Logging user activity...
